In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [5]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def get_mean_var(y_logit):
    y_logit=np.abs(y_logit-0.1)
    return np.mean(y_logit,axis=1)

def get_possibility(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=100, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
        y_logits=np.array(y_logits)
    return y_logits
import os
if not os.path.isdir('saved_models_{}'.format('acgan')):
    os.mkdir('saved_models_{}'.format('acgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('acgan'), mode='w')


from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from PIL import Image
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np

import torch.utils.data as Data
class ACGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)
        losses = ['binary_crossentropy', 'sparse_categorical_crossentropy']

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=losses,
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid, target_label = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model([noise, label], [valid, target_label])
        self.combined.compile(loss=losses,
            optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(512 * 4 * 4, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((4, 4, 512)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(256, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3,padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.channels, kernel_size=3, padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, 100)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.summary()

        img = Input(shape=self.img_shape)

        # Extract feature representation
        features = model(img)

        # Determine validity and label of the image
        validity = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes, activation="softmax")(features)

        return Model(img, [validity, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (X_test, _) = cifar10.load_data()

        # Configure inputs
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps = []
        values = []

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            # idx = np.random.randint(0, X_train.shape[0], batch_size)
            for index in range(nb_batches):
                global_step += 1
                # progress_bar.update(index)

                # get a batch of real images
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                # Image labels. 0-9
                img_labels = y_train[index * batch_size:(index + 1) * batch_size]
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # The labels of the digits that the generator tries to create an
                # image representation of
                sampled_labels = np.random.randint(0, 10, (batch_size, 1))

                # Generate a half batch of new images
                gen_imgs = self.generator.predict([noise, sampled_labels])

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(image_batch, [valid, img_labels])
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, sampled_labels])
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                # Train the generator
                g_loss = self.combined.train_on_batch([noise, sampled_labels], [valid, sampled_labels])

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (
                    epoch, global_step, d_loss[0], 100 * d_loss[3], 100 * d_loss[4], g_loss[0]))
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.mode_drop(global_step)


    def mode_drop(self, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.array([num for num in range(r) for _ in range(c)])
        gen_imgs = self.generator.predict([noise, sampled_labels])
        y_logits = get_possibility(gen_imgs)
        metrics = get_mean_var(y_logits)


        f.writelines('\n')
        f.writelines('global_step:' + str(global_step))
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in metrics)
        f.writelines('\n')
        
       

if __name__ == '__main__':
    acgan = ACGAN()
    acgan.train(epochs=50, batch_size=64, sample_interval=200)


Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): BatchNorm2d(512, eps=1e-05

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 16, 16, 64)        1792      
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 8, 8, 128)         73856     
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 8, 8, 128)         0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 8, 8, 128)         0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 8, 8, 128)         512       
__________

epoch:0 step:41 [D loss: 3.419635, acc.: 72.66%, op_acc: 17.97%] [G loss: 8.314556]
epoch:0 step:42 [D loss: 2.974202, acc.: 78.91%, op_acc: 14.84%] [G loss: 6.763103]
epoch:0 step:43 [D loss: 2.983665, acc.: 72.66%, op_acc: 17.97%] [G loss: 7.668763]
epoch:0 step:44 [D loss: 2.888028, acc.: 81.25%, op_acc: 19.53%] [G loss: 7.638926]
epoch:0 step:45 [D loss: 3.132106, acc.: 55.47%, op_acc: 25.78%] [G loss: 8.970662]
epoch:0 step:46 [D loss: 2.915157, acc.: 84.38%, op_acc: 20.31%] [G loss: 7.834035]
epoch:0 step:47 [D loss: 3.108584, acc.: 62.50%, op_acc: 13.28%] [G loss: 9.662439]
epoch:0 step:48 [D loss: 2.707020, acc.: 89.06%, op_acc: 17.97%] [G loss: 9.129260]
epoch:0 step:49 [D loss: 3.066202, acc.: 70.31%, op_acc: 20.31%] [G loss: 7.427895]
epoch:0 step:50 [D loss: 2.545398, acc.: 92.19%, op_acc: 22.66%] [G loss: 6.711745]
epoch:0 step:51 [D loss: 2.752939, acc.: 82.81%, op_acc: 17.19%] [G loss: 7.163582]
epoch:0 step:52 [D loss: 2.649256, acc.: 85.16%, op_acc: 25.00%] [G loss: 7.

epoch:0 step:141 [D loss: 2.904732, acc.: 86.72%, op_acc: 19.53%] [G loss: 6.151896]
epoch:0 step:142 [D loss: 2.661075, acc.: 85.94%, op_acc: 17.97%] [G loss: 6.784544]
epoch:0 step:143 [D loss: 2.706694, acc.: 87.50%, op_acc: 23.44%] [G loss: 5.947412]
epoch:0 step:144 [D loss: 2.806186, acc.: 82.03%, op_acc: 21.88%] [G loss: 7.406547]
epoch:0 step:145 [D loss: 2.339283, acc.: 96.88%, op_acc: 23.44%] [G loss: 7.177909]
epoch:0 step:146 [D loss: 3.036297, acc.: 60.16%, op_acc: 23.44%] [G loss: 9.538467]
epoch:0 step:147 [D loss: 2.535699, acc.: 89.84%, op_acc: 25.00%] [G loss: 8.674206]
epoch:0 step:148 [D loss: 3.044099, acc.: 67.19%, op_acc: 21.09%] [G loss: 8.802621]
epoch:0 step:149 [D loss: 2.403294, acc.: 93.75%, op_acc: 24.22%] [G loss: 9.174982]
epoch:0 step:150 [D loss: 2.740076, acc.: 89.84%, op_acc: 17.97%] [G loss: 6.299222]
epoch:0 step:151 [D loss: 3.420056, acc.: 45.31%, op_acc: 22.66%] [G loss: 9.382385]
epoch:0 step:152 [D loss: 2.590669, acc.: 92.97%, op_acc: 16.41%]

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:41: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

epoch:49 step:38627 [D loss: 0.883790, acc.: 72.66%, op_acc: 87.50%] [G loss: 1.887521]
epoch:49 step:38628 [D loss: 0.878859, acc.: 75.00%, op_acc: 88.28%] [G loss: 1.884887]
epoch:49 step:38629 [D loss: 0.960400, acc.: 71.88%, op_acc: 88.28%] [G loss: 1.909827]
epoch:49 step:38630 [D loss: 0.712269, acc.: 74.22%, op_acc: 93.75%] [G loss: 1.888825]
epoch:49 step:38631 [D loss: 0.895174, acc.: 68.75%, op_acc: 89.84%] [G loss: 1.517321]
epoch:49 step:38632 [D loss: 0.826628, acc.: 75.00%, op_acc: 89.84%] [G loss: 1.769367]
epoch:49 step:38633 [D loss: 0.905632, acc.: 75.78%, op_acc: 88.28%] [G loss: 2.461010]
epoch:49 step:38634 [D loss: 0.656327, acc.: 71.88%, op_acc: 96.88%] [G loss: 1.977928]
epoch:49 step:38635 [D loss: 0.814755, acc.: 72.66%, op_acc: 92.19%] [G loss: 1.818487]
epoch:49 step:38636 [D loss: 0.824704, acc.: 75.00%, op_acc: 84.38%] [G loss: 2.253593]
epoch:49 step:38637 [D loss: 0.874134, acc.: 71.88%, op_acc: 88.28%] [G loss: 2.180921]
epoch:49 step:38638 [D loss: 0.8

epoch:49 step:38722 [D loss: 0.828920, acc.: 71.09%, op_acc: 90.62%] [G loss: 2.167217]
epoch:49 step:38723 [D loss: 0.863198, acc.: 73.44%, op_acc: 89.84%] [G loss: 1.755235]
epoch:49 step:38724 [D loss: 0.927618, acc.: 67.97%, op_acc: 86.72%] [G loss: 1.905429]
epoch:49 step:38725 [D loss: 0.847413, acc.: 71.88%, op_acc: 92.97%] [G loss: 1.757488]
epoch:49 step:38726 [D loss: 0.889369, acc.: 77.34%, op_acc: 87.50%] [G loss: 1.696478]
epoch:49 step:38727 [D loss: 0.725022, acc.: 79.69%, op_acc: 91.41%] [G loss: 1.879056]
epoch:49 step:38728 [D loss: 0.691635, acc.: 83.59%, op_acc: 92.19%] [G loss: 1.821658]
epoch:49 step:38729 [D loss: 0.757236, acc.: 73.44%, op_acc: 91.41%] [G loss: 1.497774]
epoch:49 step:38730 [D loss: 0.850281, acc.: 73.44%, op_acc: 89.84%] [G loss: 1.770552]
epoch:49 step:38731 [D loss: 0.885789, acc.: 73.44%, op_acc: 86.72%] [G loss: 1.961086]
epoch:49 step:38732 [D loss: 0.704637, acc.: 82.81%, op_acc: 91.41%] [G loss: 1.622356]
epoch:49 step:38733 [D loss: 0.7

epoch:49 step:38817 [D loss: 0.794462, acc.: 75.78%, op_acc: 90.62%] [G loss: 1.773728]
epoch:49 step:38818 [D loss: 0.772565, acc.: 75.78%, op_acc: 86.72%] [G loss: 1.680186]
epoch:49 step:38819 [D loss: 0.881877, acc.: 71.09%, op_acc: 88.28%] [G loss: 1.330176]
epoch:49 step:38820 [D loss: 0.740803, acc.: 78.91%, op_acc: 92.19%] [G loss: 1.714223]
epoch:49 step:38821 [D loss: 0.935299, acc.: 62.50%, op_acc: 88.28%] [G loss: 1.526026]
epoch:49 step:38822 [D loss: 0.810448, acc.: 74.22%, op_acc: 89.84%] [G loss: 1.770054]
epoch:49 step:38823 [D loss: 0.798841, acc.: 75.00%, op_acc: 92.19%] [G loss: 1.834038]
epoch:49 step:38824 [D loss: 0.783387, acc.: 72.66%, op_acc: 86.72%] [G loss: 1.738454]
epoch:49 step:38825 [D loss: 0.786853, acc.: 73.44%, op_acc: 89.06%] [G loss: 1.766841]
epoch:49 step:38826 [D loss: 0.870150, acc.: 75.00%, op_acc: 89.06%] [G loss: 1.592533]
epoch:49 step:38827 [D loss: 0.817480, acc.: 69.53%, op_acc: 92.97%] [G loss: 1.828464]
epoch:49 step:38828 [D loss: 1.0

epoch:49 step:38911 [D loss: 0.892219, acc.: 72.66%, op_acc: 89.84%] [G loss: 1.784256]
epoch:49 step:38912 [D loss: 0.917074, acc.: 78.12%, op_acc: 88.28%] [G loss: 1.887246]
epoch:49 step:38913 [D loss: 0.685319, acc.: 75.78%, op_acc: 95.31%] [G loss: 1.662338]
epoch:49 step:38914 [D loss: 0.734752, acc.: 86.72%, op_acc: 88.28%] [G loss: 1.779258]
epoch:49 step:38915 [D loss: 0.819418, acc.: 75.78%, op_acc: 91.41%] [G loss: 1.970467]
epoch:49 step:38916 [D loss: 0.792320, acc.: 78.12%, op_acc: 89.06%] [G loss: 1.672748]
epoch:49 step:38917 [D loss: 1.005119, acc.: 70.31%, op_acc: 85.16%] [G loss: 1.731167]
epoch:49 step:38918 [D loss: 0.781751, acc.: 78.91%, op_acc: 87.50%] [G loss: 2.159115]
epoch:49 step:38919 [D loss: 0.929744, acc.: 71.09%, op_acc: 89.06%] [G loss: 1.955476]
epoch:49 step:38920 [D loss: 0.921301, acc.: 70.31%, op_acc: 89.06%] [G loss: 1.498377]
epoch:49 step:38921 [D loss: 0.780804, acc.: 74.22%, op_acc: 91.41%] [G loss: 1.749640]
epoch:49 step:38922 [D loss: 0.8

epoch:49 step:39005 [D loss: 0.887736, acc.: 80.47%, op_acc: 83.59%] [G loss: 1.971002]
epoch:49 step:39006 [D loss: 0.659061, acc.: 75.00%, op_acc: 93.75%] [G loss: 1.899242]
epoch:49 step:39007 [D loss: 0.644161, acc.: 84.38%, op_acc: 90.62%] [G loss: 1.864487]
epoch:49 step:39008 [D loss: 0.830269, acc.: 72.66%, op_acc: 91.41%] [G loss: 1.615548]
epoch:49 step:39009 [D loss: 0.741250, acc.: 76.56%, op_acc: 89.84%] [G loss: 1.701666]
epoch:49 step:39010 [D loss: 0.818444, acc.: 77.34%, op_acc: 85.94%] [G loss: 1.687667]
epoch:49 step:39011 [D loss: 0.869428, acc.: 79.69%, op_acc: 85.94%] [G loss: 1.859863]
epoch:49 step:39012 [D loss: 0.884936, acc.: 72.66%, op_acc: 89.84%] [G loss: 1.655714]
epoch:49 step:39013 [D loss: 0.698425, acc.: 69.53%, op_acc: 92.97%] [G loss: 1.696531]
epoch:49 step:39014 [D loss: 0.820893, acc.: 78.91%, op_acc: 89.06%] [G loss: 1.734760]
epoch:49 step:39015 [D loss: 0.919208, acc.: 66.41%, op_acc: 91.41%] [G loss: 1.708467]
epoch:49 step:39016 [D loss: 0.7